# Problem description

- We have a set of $J={1,...,n}$ of electric vehicle charging demands to be scheduled on a set of chargers.
- At each time, a charger can only charge one vehicle, and a vehicle can only be charged by one charger.
- Each charger $i$ delivers a constant power $w_i$ (kW).
- The total power that can be delivered by all chargers simultaneously must not exceed $w_G$ (kW), the power grid capacity.
- The charging demand of each vehicle $j$ is characterized by an arrival time $r_j$ , a departure time $d_j$ , and required energy $e_j$ (kWh).
- Each charging demand j has to be assigned to one charger, and its energy requirement must be fulfilled before the departure.
  - The vehicle $j$ uninterruptedly occupies the charger, and the parking space, from its arrival time $r_j$ to its departure time $d_j$ and cannot be moved or unplugged during this time.
  - The vehicle can charge preemptively.
  - Even if j completes charging before dj , it still occupies the charger until it departs.
- We divide the scheduling time horizon $H$ into $T$ time slots of equal length $\tau$.
  - $r_j$ and $d_j$ are multiple of $\tau$.
- We assume linear charging time of vehicles, meaning that the charging time $p_{ij}$ of vehicle $j$ on a charger $i$ is equal to $\frac{e_j}{w_i}$.
  - $p_{ij}$ is the number of time slots needed to satisfy $j$ rounded to the nearest integer.

The objective is to find a feasible schedule with the minimum grid capacity $w_G$.

# Identical charges
instance of the electric vehicle charging problem with identical chargers, each delivering a constant power $w$ (kW), and the number of chargers is not fixed in advance. we address the problem of the minimum number of chargers needed to charge all vehicles before minimizing the capacity of the grid.

## Minimum number of chargers

- Charging a vehicle j requires the assignment of an available charger to that vehicle for an uninterrupted period length $[rj , dj)$.
  - We assume that the energy demand of vehicle j can be fulfilled during the plugging time interval
- This problem is equivalent to finding the minimum number of machines that can handle all jobs in the fixed interval scheduling problem.

Let $z$ be the maximum number of charging demands that overlap at the same time. This means that at some moment in time, there are $z$ different demands that all require charging simultaneously.
The minimum number of identical chargers needed to plug n electric vehicles is equal to $z$.

In [15]:
import numpy as np
import gurobipy as gb
import math
import random

class ChargingDemand:
    def __init__(self, r, d, e):
        """
        :param r: arrival time
        :param d: departure time
        :param e: energy demand
        """
        self.r = r
        self.d = d
        self.e = e
    
    def __repr__(self):
        return f"ChargingDemand(r={self.r}, d={self.d}, e={self.e})"

        
def minimum_number_of_identical_chargers(J, num_time_slots, beginning_time_slot):
    J.sort(key=lambda x: x.r)
    
    demands = np.zeros((1, num_time_slots))
    
    for j_i, j in enumerate(J):
        r = j.r - beginning_time_slot
        d = j.d - beginning_time_slot
        
        # Check if there's room in this row
        for i in range(demands.shape[0]):
            if demands[i, r] == 0:
                demands[i, r:d] = j_i+1
                break
            else:
                # Add a new row
                demands = np.vstack((demands, np.zeros(num_time_slots)))
                demands[-1, r:d] = j_i+1
                break
    
    return demands.shape[0], demands

# Same data as in the Example 3.1
J = [
    ChargingDemand(10, 13, 20),
    ChargingDemand(10, 13, 20),
    ChargingDemand(8, 10, 20),
    ChargingDemand(10, 13, 20),
    ChargingDemand(10, 13, 20),
    ChargingDemand(9, 12, 30)
]

beginning_time_slot = min(J, key=lambda x: x.r).r
H = [i for i in range(
    max(J, key=lambda x: x.d).d - beginning_time_slot,
    )] # Time slots
m, demands = minimum_number_of_identical_chargers(J, len(H), beginning_time_slot)

assert m == 5
m, demands

(5,
 array([[1., 1., 3., 3., 3.],
        [0., 2., 2., 2., 0.],
        [0., 0., 4., 4., 4.],
        [0., 0., 5., 5., 5.],
        [0., 0., 6., 6., 6.]]))

### Optimal algorithm


## Minimum grid capacity

Problem description:
- We assume that $z$ chargers are available.
- The objective here is to determine the charging times of the demands that minimize the grid capacity required to charge all vehicles to their desired energy.

Problem formulation:
- We define a binary decision variable $x_{jt}$ , for each $j \in J$ , and $t \in H$, that takes value one if vehicle $j$ is charging at time slot $t$.
- Let $w_G$ be a non-negative continuous variable that represents the grid capacity value.
- $w_G$ can be expressed as a max function of the variables $x_{jt} , i.e., $w_G = max_{t \in H} \sum_{j=1}^{n} wx_{jt}$.

Therefore, we have the following integer linear program:
- Objective function:
  - Minimize $w_G$
- Subject to:
  - $\sum_{t=1}^{T} x_{ij} = p_j        \forall j \in J$, ensure that the charging demand of vehicle $j$ is satisfied.
  - $\sum_{j=1}^{n} w x_{ij} \leq w_G   \forall t \in H$, calculate the total power delivered to vehicles at each time slot $t$.
  - $x_{ij} = 0                         \forall j \in J, t \not\in [r_j, d_j)$, guarantee that each vehicle $j$ will not be charged before its arrival time $r_j$ nor after its departure time $d_j$

In [16]:
P = [] # Charging time of vehicles
w = 10 # Power of each charger
for j in J:
    P.append(j.e/w)

minimum_grid_capacity = gb.Model()

x = minimum_grid_capacity.addVars(len(J), len(H), vtype=gb.GRB.BINARY, name="x")
wG = minimum_grid_capacity.addVar(vtype=gb.GRB.CONTINUOUS, name="wG")

# Add constraint 2
for j in range(len(J)):
    minimum_grid_capacity.addConstr(
        gb.quicksum(x[j, t] for t in range(len(H))) == P[j],
        name=f"charging_demand_{j}"
    )
    
# Add constraint 3
for t in range(len(H)):
    minimum_grid_capacity.addConstr(
        gb.quicksum(w * x[j, t] for j in range(len(J))) <= wG,
        name=f"total_power_{t}"
    )
    
# Add constraint 4
for j in range(len(J)):
    for t in range(len(H)):
        if  t < J[j].r - beginning_time_slot or \
            t > J[j].d - beginning_time_slot:
            minimum_grid_capacity.addConstr(
                x[j, t] == 0,
                name=f"not_charged_if_not_parked_{j}_{t}"
            )
        
minimum_grid_capacity.setObjective(wG, gb.GRB.MINIMIZE)

In [17]:
minimum_grid_capacity.optimize()

if minimum_grid_capacity.status == gb.GRB.OPTIMAL:
    optim = int(minimum_grid_capacity.objVal)
    print("\nOptimal grid capacity:", optim)
    assert optim == 40
else:
    print("No feasible solution found.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 22 rows, 31 columns and 76 nonzeros
Model fingerprint: 0xf9b752f4
Variable types: 1 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 3e+00]
Found heuristic solution: objective 50.0000000
Presolve removed 12 rows and 11 columns
Presolve time: 0.00s
Presolved: 10 rows, 20 columns, 41 nonzeros
Variable types: 0 continuous, 20 integer (19 binary)

Root relaxation: objective 3.333333e+01, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   3

### Polynomial algorithm

minimizing the value of wG is equivalent to minimizing the number of chargers to be activated simultaneously at each time.
To solve this optimization problem, we consider its decision problem, i.e., given an integer value
m̃, deciding whether a feasible preemptive schedule exists that satisfies n charging demands by
simultaneously activating at most m̃ chargers.

In [18]:
def get_max_flow(J, m_hat, S_V, intervals, interval_to_i):
    # Calculate the matrix binding vehicles to intervals
    V_I = np.zeros((len(J), len(intervals)))
    available_capacity = []
    
    for interval in intervals:
        available_capacity.append(m_hat * (interval[1] - interval[0]))
    
    for i in range(len(S_V)):
        # for each vehicle demand I have to put how much time this stays plugged in
        residual = S_V[i]
        
        interesting_intervals = []
        for interval in intervals:
            if interval[0] >= J[i].r and interval[1] <= J[i].d:
                interesting_intervals.append(interval)
        
        for interesting_interval in interesting_intervals:
            if residual == 0:
                break
            slot_duration = interesting_interval[1] - interesting_interval[0]
            interval_id = interval_to_i[interesting_interval[0]]
            how_much_time = min(residual, slot_duration)
            available_capacity[interval_id] -= how_much_time
            if available_capacity[interval_id] < 0:
                continue
            V_I[i][interval_id] = how_much_time
            residual -= how_much_time
    
    # Calculate interval to sink mapping
    I_P = []
    for interval in intervals:
        I_P.append(m_hat * (interval[1] - interval[0]))
    
    # Now I have to calculate the maximum flow of power from the interval to the sink
    max_flow = 0
    for col_i in range(V_I.shape[1]):
        # Sum this column
        col_sum = sum(V_I[:, col_i])
        max_flow += min(col_sum, I_P[col_i])
        
    return max_flow + residual

def max_chargers_activated_simultaneously(J, m):
    # Calculate source to vehicle mapping
    S_V = []
    for j in J:
        S_V.append(j.e/10)
    P = sum(S_V)
    
    # Get each unique time slot
    L = []
    for j in J:
        if j.r not in L:
            L.append(j.r)
        if j.d not in L:
            L.append(j.d)
    L.sort()
    
    # Calculate each possible interval
    intervals = []
    interval_to_i = {}
    for i in range(len(L)-1):
        intervals.append((L[i], L[i+1])) # arrival, departure
        interval_to_i[L[i]] = i
    
    # Now I have to do the binary search on this array containing the optimal number of chargers
    m_hats = [ i for i in range(1, m+1) ]
    m_hat = -1
    
    i = math.ceil(len(m_hats) / 2)
    max_flow = -1
    while max_flow != P:
        m_hat = m_hats[i]
        max_flow = get_max_flow(J, m_hat, S_V, intervals, interval_to_i)
        
        if max_flow < P:
            m_hats = m_hats[i+1:]
        elif max_flow > P:
            m_hats = m_hats[:i]
        else:
            break
        
        i = math.ceil(len(m_hats) / 2)
    
    return m_hat
    
max_chargers_activated_simultaneously(J, m)

4

# Distinct types of chargers

- General case with $k$ types of chargers, where each type $i$ delivers a
constant power of $w_i$ (kW).
- The charging time of demand $j$ on a charger of type $i$ is equal to $p_{ij} = \frac{e_j}{w_i \tay}$.
- $k$ types of chargers are indexed in the non-decreasing order of their power.
- We assume that there exists at least one type of chargers on which each charging demand $j$ can be satisfied.

## Minimum number of chargers

Since a type k of chargers can satisfy all charging demands, considering only chargers of type k can lead to high power consumption
when minimizing the capacity of the grid.
it is then possible to replace each
charger of type k with a charger with a lower power
The chargers substitution operation has
a complexity O(n2 ).


In [19]:
def required_charging_power(demand: ChargingDemand):
    # duration = demand.d - demand.r
    # return (demand.e + duration - 1) // duration  # ceil division
    return demand.e

def minimum_number_of_diverse_chargers(J, num_time_slots, beginning_time_slot,
                                       charger_types):
    J.sort(key=lambda x: x.r)
    
    demands = np.zeros((1, num_time_slots))
    charger_powers = [required_charging_power(J[0])]
    
    for j_i, j in enumerate(J):
        r = j.r - beginning_time_slot
        d = j.d - beginning_time_slot
        
        # Check if there's room in this row
        for i in range(demands.shape[0]):
            charger_power = required_charging_power(j)
            assert charger_power in charger_types, f"Charger power {charger_power} not in available types" # This should never happen
            add_new_row = False
            
            if demands[i, r] == 0:
                # Check if this charger can serve enough power
                if charger_powers[i] <= charger_power:
                    demands[i, r:d] = j_i+1
                else:
                    add_new_row = True
                break
            else:
                add_new_row = True
            
            if add_new_row:
                # Add a new row
                demands = np.vstack((demands, np.zeros(num_time_slots)))
                demands[-1, r:d] = j_i+1
                charger_powers.append(charger_power)
                break
    
    return demands.shape[0], demands, charger_powers

charger_types = [10, 20, 30]
m, demands, charger_powers = minimum_number_of_diverse_chargers(J, len(H), beginning_time_slot, charger_types)
m, demands, charger_powers

(5,
 array([[1., 1., 3., 3., 3.],
        [0., 2., 2., 2., 0.],
        [0., 0., 4., 4., 4.],
        [0., 0., 5., 5., 5.],
        [0., 0., 6., 6., 6.]]),
 [20, 30, 20, 20, 20])

## Minimum grid capacity

This is a test of a scheduling using the example given.

In [20]:
X = np.array([
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 0, 0, 1],
    [0, 0, 0, 1, 0],
])

Y = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [1, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1]
])

P = np.array([
    [1, 2, 1, 1, 1],
    [1, 3, 2, 2, 2],
    [1, 2, 1, 1, 1],
    [1, 2, 1, 1, 1],
    [1, 2, 1, 1, 1],
    [1, 2, 1, 1, 1]
])

W = np.array([[30, 10, 20, 20, 20]])

W @ Y @ X

array([[20, 30, 30, 30, 20]])

In [21]:
W = [30, 10, 20, 20, 20]

P = np.zeros((len(J), len(W)))
for j in range(P.shape[0]):
    for l in range(P.shape[1]):
        P[j, l] = math.ceil(J[j].e / W[l])

minimum_grid_capacity2 = gb.Model()

x = minimum_grid_capacity2.addVars(len(J), len(H), vtype=gb.GRB.BINARY, name="x")
y = minimum_grid_capacity2.addVars(len(W), len(J), vtype=gb.GRB.BINARY, name="y")
z = minimum_grid_capacity2.addVars(len(J), len(W), len(H), vtype=gb.GRB.BINARY, name="z")
wG = minimum_grid_capacity2.addVar(vtype=gb.GRB.CONTINUOUS, name="wG")

# Add constraints 8
for j in range(len(J)):
    minimum_grid_capacity2.addConstr(
        gb.quicksum(x[j, t] for t in range(len(H))) == gb.quicksum(P[j, l] * y[l, j] for l in range(len(W))),
        name=f"charging_demand_{j}"
    )
    
# Add constraints 9
for j in range(len(J)):
    minimum_grid_capacity2.addConstr(
        gb.quicksum(y[l, j] for l in range(len(W))) == 1,
        name=f"charger_type_{j}"
    )

# Add constraints 11
for j in range(len(J)):
    for t in range(len(H)):
        if  t < J[j].r - beginning_time_slot or \
            t > J[j].d - beginning_time_slot:
            minimum_grid_capacity2.addConstr(
                x[j, t] == 0,
                name=f"not_charged_if_not_parked_{j}_{t}"
            )

for j in range(len(J)):
    for l in range(len(W)):
        for t in range(len(H)):
            # Add constraints 12
            minimum_grid_capacity2.addConstr(
                z[j, l, t] >= x[j, t] + y[l, j] - 1,
                name=f"z_geq_{j}_{l}_{t}"
            )
            
            # Add constraints 13
            minimum_grid_capacity2.addConstr(
                z[j, l, t] <= x[j, t],
                name=f"z_leq_x_{j}_{l}_{t}"
            )
            
            # Add constraints 14
            minimum_grid_capacity2.addConstr(
                z[j, l, t] <= y[l, j],
                name=f"z_leq_y_{j}_{l}_{t}"
            )
            
# Add constraints 15
for t in range(len(H)):
    minimum_grid_capacity2.addConstr(
        gb.quicksum(W[l] * z[j, l, t] for j in range(len(J)) for l in range(len(W))) <= wG,
        name=f"w_{l}_z_{j}_{l}_{t}_leq_wG"
    )

# Add constraints 16
# m = []
# for l in range(len(W)):
#     minimum_grid_capacity2.addConstr(
#         gb.quicksum(y[l, j] for j in range(len(J))) <= m[l],
#         name=f"y_{l}_leq_m_{l}"
#     )
        
minimum_grid_capacity2.setObjective(wG, gb.GRB.MINIMIZE)

In [22]:
minimum_grid_capacity2.optimize()

if minimum_grid_capacity2.status == gb.GRB.OPTIMAL:
    optim = int(minimum_grid_capacity2.objVal)
    print("\nOptimal grid capacity:", optim)
    assert optim == 30
else:
    print("No feasible solution found.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 478 rows, 211 columns and 1306 nonzeros
Model fingerprint: 0xa325b3ff
Variable types: 1 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 60.0000000
Presolve removed 176 rows and 66 columns
Presolve time: 0.01s
Presolved: 302 rows, 145 columns, 823 nonzeros
Variable types: 0 continuous, 145 integer (144 binary)

Root relaxation: objective 0.000000e+00, 46 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

    

### Solving methods using heuristics

In [23]:
def calculate_lb(J, W):
    d = [j.d for j in J]
    r = [j.r for j in J]
    e = [j.e for j in J]
    
    # First term: ceil(total_energy / total_available_time)
    total_energy = sum(e)
    total_time = max(d) - min(r)
    e1 = np.ceil(total_energy / total_time) if total_time > 0 else 0
    
    # Second term: find minimal charger type >= max(ej/(dj-rj))
    e2_values = [ej / (dj - rj) for ej, dj, rj in zip(e, d, r) if dj > rj]
    e2 = max(e2_values) if e2_values else 0
    
    # Find smallest charger type >= e2
    sorted_W = sorted(W)
    for w in sorted_W:
        if w >= e2:
            wl = w
            break
    else:  # No charger found (shouldn't happen)
        wl = sorted_W[-1]
    
    return max(e1, wl)

def heuristic_grid_capacity_minimization(J, W, num_time_slots, beginning_time_slot):
    W = np.array(W)
    J = sorted(J, key=lambda x: (x.d, -x.e, x.r))
    
    wgT = np.zeros(num_time_slots)
    sigmas = np.zeros(len(J))
    schedule = np.zeros((len(J), num_time_slots))
    
    lb = calculate_lb(J, W)
    
    wG = lb
    for j_i, j in enumerate(J):
        # --- Line 5
        current_max = max(lb, wG)  # Current power threshold
        
        # Get time interval [rj, dj)
        start = j.r - beginning_time_slot
        end = j.d - beginning_time_slot
        interval = wgT[start:end]  # Slicing is [start, end)
        
        # --- Line 6
        
        # Calculate available slots (b)
        mask = interval < current_max
        b = np.sum(mask)
        
        # Calculate wG_b (max power in available slots)
        if b > 0:
            wG_b = np.max(interval[mask])
        else:
            wG_b = 0  # No available slots

        # --- Lines 7-8 ---
        if int(j.e/b + wG_b) <= current_max:
            mask = W <= math.ceil(math.ceil(j.e/b)/10)*10
            charger_type = np.max(W[mask])
            
            sigmas[j_i] = charger_type
        else:
            sigmas[j_i] = math.ceil(math.ceil(j.e/b)/10)*10 # Wlj undefined in the paper. This gives similar numbers.
            
        # Schedule the charging according to algorithm 4
        wG, wgT, times = power_allocation(sigmas, j, j_i, start, end, wgT, current_max, wG)
        
        # use times to build the schedule
        for t in times:
            schedule[j_i][t] = 1
        
        pass
        
    return wG, wgT, sigmas, schedule

def power_allocation(sigmas, j, j_i, start, end, wgT, current_max, wG):
    p = j.e / sigmas[j_i]
        
    H1 = []
    H2 = []
    for t in range(start, end):
        if wgT[t] < current_max:
            H1.append(t)
        else:
            H2.append(t)
    
    H2.sort(key=lambda x: wgT[x])
    
    times = []
    
    while p > 0:
        if H1:
            t = H1.pop(0)
        elif H2:
            t = H2.pop(0)
        else:
            raise Exception("No available time slots")

        wgT[t] += sigmas[j_i]
        p -= 1
        
        if wgT[t] > wG:
            wG = wgT[t]
            
        times.append(t)
            
    return wG, wgT, times

wG, wgT, sigmas, schedule = heuristic_grid_capacity_minimization(J, W, len(H), beginning_time_slot)
wG, wgT, sigmas, schedule

(40.0,
 array([10., 20., 30., 30., 40.]),
 array([10., 10., 10., 10., 20., 20.]),
 array([[1., 1., 0., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 1., 1., 0.],
        [0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.]]))

### Iterated Local Search (ILS)

In [24]:
def calculate_wG(sigmas, schedule):
    instant_powers = sigmas @ schedule
    
    return max(instant_powers)

class Solution:
    def __init__(self, J, sigmas, schedule, beginning_time_slot, available_charger_types):
        self.J = J
        self.sigmas = sigmas
        self.schedule = schedule
        self.beginning_time_slot = beginning_time_slot
        self.available_charger_types = available_charger_types
        self.calculate_wG()
    
    def calculate_wG(self):
        self.wG = calculate_wG(self.sigmas, self.schedule)
    
    def copy(self):
        """Creates a deep copy of the solution for independent modification."""
        # sigmas is assumed to be constant and can be shallow copied (referenced)
        # schedule needs a deep copy as it's modified
        copied_schedule = self.schedule.copy()
        return Solution(self.J, self.sigmas, copied_schedule, self.beginning_time_slot, self.available_charger_types)

    def __repr__(self):
        return f"Solution(wG={self.wG:.4f})"

In [25]:
def Perturbation(solution_to_perturb: Solution) -> Solution:
    """
    Applies a perturbation to the given solution.
    Based on: "Random selection: select a random demand j and assigned it to a random type."
    """
    new_solution = solution_to_perturb.copy() # Work on a copy

    num_types, num_demands = new_solution.schedule.shape

    # 1. Select a random demand j
    random_demand_idx = random.randint(0, num_demands - 1)

    selected_row = new_solution.schedule[random_demand_idx]
    # Now that we have the row, we have to swap some 0 and 1 while the car is parked
    beginning = new_solution.J[random_demand_idx].r - new_solution.beginning_time_slot
    end = new_solution.J[random_demand_idx].d - new_solution.beginning_time_slot
    selected_row_roi = selected_row[beginning:end]
    
    # Now swap some items in the selected row
    def swap_items_in_row(arr):
        arr_copy = arr.copy()

        indices_of_zeros = np.where(arr_copy == 0)[0]
        indices_of_ones = np.where(arr_copy == 1)[0]

        # Check if both 0s and 1s exist in the array
        if len(indices_of_zeros) == 0 or len(indices_of_ones) == 0:
            # print("Warning: Array does not contain both 0s and 1s. No swap performed.")
            return arr_copy # Return the copy of the original

        # Randomly select one index for a 0 and one index for a 1
        idx_zero_to_swap = np.random.choice(indices_of_zeros)
        idx_one_to_swap = np.random.choice(indices_of_ones)

        # Perform the swap
        arr_copy[idx_zero_to_swap] = 1
        arr_copy[idx_one_to_swap] = 0
        
        return arr_copy

    selected_row_roi = swap_items_in_row(selected_row_roi)
    selected_row[beginning:end] = selected_row_roi
    new_solution.schedule[random_demand_idx] = selected_row
    
    # Now swap some charger types
    to_replace = random.randint(0, len(solution_to_perturb.sigmas) - 1)
    with_what = random.choice(solution_to_perturb.available_charger_types)
    new_solution.sigmas[to_replace] = with_what
    
    new_solution.calculate_wG()
    return new_solution

In [26]:
def LocalSearch(current_solution: Solution) -> Solution:
    return current_solution

In [27]:
def iterated_local_search(
    initial_solution: Solution,
    pert0: float,       # Initial perturbation strength/level
    pert_max_abs: float, # Absolute maximum perturbation strength/level
    iter_max: int,      # Max iterations without improvement before increasing pert
    r_accept: float,    # Parameter for acceptance probability of non-improving solutions
    p0_accept: float = 1.0 # Acceptance probability base for non-improving solutions
) -> Solution:
    """
    Implements the Iterated Local Search algorithm (Algorithm 5).
    Assumes f(S) means solution.wG and lower is better.
    """
    iter_count = 0
    current_pert_strength = pert0

    # S, S', S* are Solution objects
    S = initial_solution.copy()
    S_prime = initial_solution.copy()  # S' in pseudocode (base for perturbation)
    S_star = initial_solution.copy()   # S* in pseudocode (best solution found)

    print(f"ILS Start: S0.wG={initial_solution.wG:.4f}, pert0={pert0}, pert_max_abs={pert_max_abs}, iter_max={iter_max}, r={r_accept}")

    main_loop_count = 0
    while current_pert_strength < pert_max_abs and main_loop_count < 1000:
        main_loop_count += 1
        print(f"\nILS Main Loop {main_loop_count}: current_pert_strength={current_pert_strength:.2f}, S_star.wG={S_star.wG:.4f}")

        # Line 4: Choose p, number of perturbation steps
        max_p_val = int(round(current_pert_strength))
        if max_p_val < 1:
            max_p_val = 1 # Apply at least one perturbation if pert_strength is very small but > 0
        
        p_perturb_steps = random.randint(1, max_p_val)
        print(f"  Perturbing {p_perturb_steps} times (max_p_val from strength {current_pert_strength:.2f})")

        # Lines 5-7: Apply Perturbation p times starting from S_prime
        S_perturbed = S_prime.copy() # Start perturbation from S_prime
        for _ in range(p_perturb_steps):
            S_perturbed = Perturbation(S_perturbed)
        print(f"  After Perturbation: S_perturbed.wG={S_perturbed.wG:.4f}")

        # Line 8: Apply Local Search
        S_after_ls = LocalSearch(S_perturbed)
        print(f"  After LocalSearch: S_after_ls.wG={S_after_ls.wG:.4f}")

        # Lines 9-11: Acceptance Criterion
        if S_after_ls.wG < S_star.wG:
            print(f"  New best found! S_after_ls.wG ({S_after_ls.wG:.4f}) < S_star.wG ({S_star.wG:.4f})")
            S_prime = S_after_ls.copy()
            S_star = S_after_ls.copy()
            iter_count = 0
            current_pert_strength = pert0 # Reset perturbation strength
        else:
            # Lines 12-15: Probabilistic acceptance for non-improving solution
            u = random.uniform(0, 1)
            # Sticking to pseudocode:
            if iter_count == 0 and r_accept > 0: # Handle iter_count-1 = -1
                 acceptance_prob = p0_accept * (1.0 / r_accept if r_accept != 0 else float('inf'))
            elif r_accept == 0 and iter_count > 0: # Avoid 0^positive_power
                acceptance_prob = 0
            else:
                 acceptance_prob = p0_accept * (r_accept ** (iter_count - 1))

            if u < acceptance_prob:
                print(f"  Accepted non-improving solution: S_after_ls.wG ({S_after_ls.wG:.4f}), u={u:.2f} < prob={acceptance_prob:.2f}")
                S_prime = S_after_ls.copy()
                iter_count += 1
            else:
                print(f"  Rejected non-improving solution: S_after_ls.wG ({S_after_ls.wG:.4f}), u={u:.2f} >= prob={acceptance_prob:.2f}")

        # Lines 17-19: Increase perturbation strength if stuck
        if iter_count >= iter_max:
            print(f"  iter_count ({iter_count}) >= iter_max ({iter_max}). Increasing perturbation.")
            iter_count = 0
            current_pert_strength += pert0
            S_prime = S_star.copy() # Restart perturbation from the best known solution

    print(f"ILS Finished. Best solution S_star.wG={S_star.wG:.4f}")
    return S_star

available_charger_types = [10, 20, 30]
initial_solution = Solution(J, sigmas, schedule, beginning_time_slot, available_charger_types)
pert0 = 0.1
pertMax = 0.5
iterMax = 100
r = 0.1
solution = iterated_local_search(initial_solution, pert0, pertMax, iterMax, r)
print("Initial solution:", initial_solution.wG)
print("New solution:", solution.wG)

ILS Start: S0.wG=40.0000, pert0=0.1, pert_max_abs=0.5, iter_max=100, r=0.1

ILS Main Loop 1: current_pert_strength=0.10, S_star.wG=40.0000
  Perturbing 1 times (max_p_val from strength 0.10)
  After Perturbation: S_perturbed.wG=50.0000
  After LocalSearch: S_after_ls.wG=50.0000
  Accepted non-improving solution: S_after_ls.wG (50.0000), u=0.91 < prob=10.00

ILS Main Loop 2: current_pert_strength=0.10, S_star.wG=40.0000
  Perturbing 1 times (max_p_val from strength 0.10)
  After Perturbation: S_perturbed.wG=50.0000
  After LocalSearch: S_after_ls.wG=50.0000
  Accepted non-improving solution: S_after_ls.wG (50.0000), u=0.93 < prob=1.00

ILS Main Loop 3: current_pert_strength=0.10, S_star.wG=40.0000
  Perturbing 1 times (max_p_val from strength 0.10)
  After Perturbation: S_perturbed.wG=80.0000
  After LocalSearch: S_after_ls.wG=80.0000
  Rejected non-improving solution: S_after_ls.wG (80.0000), u=0.44 >= prob=0.10

ILS Main Loop 4: current_pert_strength=0.10, S_star.wG=40.0000
  Perturbi